In [ ]:
# Exercise 1: Retrieving IP addresses

import socket

def get_ip_address(website_url):
    try:
        ip_address = socket.gethostbyname(website_url)
        print(f"The IP address of {website_url} is {ip_address}")
    except socket.gaierror:
        print(f"Unable to get the IP address for {website_url}")


website = input("Enter the website URL (without the 'https://'): ")
get_ip_address(website)

# The IP address of google.com is 216.58.213.14
# The IP address of learn.gold.ac.uk is 52.169.178.98
# The IP address of github.com is 20.26.156.215

The IP address of github.com is 20.26.156.215


In [ ]:
# Exercise 2: Trace Route

import subprocess

def tracert(domain):
    try:
        result = subprocess.run(["tracert", domain] , capture_output=True, text=True)
        print(result.stdout)
    except FileNotFoundError:
        print("Trace route command not found. Make sure it's available on your system.")
    except Exception as e:
        print(f"An error occurred: {e}")

domain = input("Enter the website or IP address for trace route: ")
tracert(domain)


Tracing route to youtube-ui.l.google.com [172.217.169.14]
over a maximum of 30 hops:

  1     1 ms     1 ms     1 ms  10.32.128.1 
  2    61 ms     9 ms     3 ms  gw-vlan2593.e2.hex.ask4.net [213.143.8.25] 
  3    18 ms     4 ms     3 ms  core1.lond2.uk.ask4.net [213.143.0.150] 
  4    30 ms     4 ms     3 ms  core2.lond2.uk.ask4.net [213.143.0.145] 
  5     2 ms     2 ms     3 ms  ask4-google-10ge-1.hex.ask4.net [213.143.6.5] 
  6     4 ms     4 ms     3 ms  209.85.249.149 
  7     5 ms     5 ms     3 ms  209.85.241.95 
  8     3 ms     3 ms     5 ms  lhr25s26-in-f14.1e100.net [172.217.169.14] 

Trace complete.



: 

# Exercise 3 and 4: Analysis and Bottlenecks

1. github.com

Tracing route to github.com [20.26.156.215]
over a maximum of 30 hops:

  1     1 ms     2 ms     1 ms  10.32.128.1 
  2     4 ms     3 ms     3 ms  gw-vlan2593.e2.hex.ask4.net [213.143.8.25] 
  3     3 ms    12 ms     2 ms  core2.lond2.uk.ask4.net [213.143.0.152] 
  4     2 ms     2 ms     2 ms  fourhundredgigabitethernet33-1.core1.lond1.uk.ask4.net [213.143.0.182] 
  5    85 ms    12 ms     3 ms  ae77-0.ier06.lon32.ntwk.msn.net [104.44.47.167] 
  6     4 ms     4 ms     8 ms  ae35-0.icr02.lon22.ntwk.msn.net [51.10.15.217] 
  7     5 ms     4 ms     *     be-122-0.ibr02.lon22.ntwk.msn.net [104.44.21.97] 
  8     *        *        4 ms  51.10.35.48 
  9     5 ms     5 ms     6 ms  51.10.16.206 
 10     *        *        *     Request timed out.
 11     *        *        *     Request timed out.
 12     *        *        *     Request timed out.
 13     *        *        *     Request timed out.
 14     3 ms     4 ms     3 ms  20.26.156.215 

Trace complete.

The traceroute to github.com begins with a local gateway (10.32.128.1) and quickly moves to an ISP gateway at hop 2 (gw-vlan2593.e2.hex.ask4.net [213.143.8.25]). These first two hops indicate that all traffic from my network uses the same initial route.

After these initial hops, the route continues through core routers in London (hops 3 and 4) with addresses such as core2.lond2.uk.ask4.net [213.143.0.152] and fourhundredgigabitethernet33-1.core1.lond1.uk.ask4.net [213.143.0.182]. However, the route for github.com then diverges into additional hops, including one with a notably high latency measurement (85 ms at hop 5) and several hops (hops 10 to 13) that time out.

Ultimately, the trace concludes at hop 14 with the destination IP 20.26.156.215. The extra hops and occasional high latency suggest that the route to GitHub is more complex. This complexity might be due to different routing policies or the use of additional security measures within the network that hosts GitHub services.

2. google.com

Tracing route to www.google.com [142.250.187.228]
over a maximum of 30 hops:

  1     3 ms     1 ms     1 ms  10.32.128.1 
  2     8 ms     3 ms     3 ms  gw-vlan2593.e2.hex.ask4.net [213.143.8.25] 
  3     4 ms     2 ms     3 ms  core1.lond2.uk.ask4.net [213.143.0.150] 
  4     7 ms     2 ms     2 ms  core2.lond2.uk.ask4.net [213.143.0.145] 
  5     2 ms     2 ms     2 ms  ask4-google-10ge-1.hex.ask4.net [213.143.6.5] 
  6     3 ms     6 ms     3 ms  209.85.248.229 
  7     9 ms     3 ms     2 ms  142.251.54.47 
  8     3 ms     3 ms     3 ms  lhr25s34-in-f4.1e100.net [142.250.187.228] 

Trace complete.

The traceroute for google.com starts similarly, with the same local gateway and ISP gateway as github.com. However, after these initial hops, the route to google.com follows a more streamlined path. After passing through the core routers (hops 3 and 4), the packet reaches a specialised node labeled ask4-google-10ge-1.hex.ask4.net [213.143.6.5] at hop 5, indicating that traffic for Google is directed to a node dedicated to handling Google requests.

The path continues with a few more hops (6 and 7) that maintain low and consistent round-trip times, typically between 1 and 9 milliseconds. Finally, the traceroute ends at hop 8 with the destination IP 142.250.187.228. The shorter path and the minimal delays imply an optimised, well-managed route for Google’s network traffic, resulting in efficient and fast connectivity.

3. youtube.com

Tracing route to youtube-ui.l.google.com [172.217.169.14]
over a maximum of 30 hops:

  1     1 ms     1 ms     1 ms  10.32.128.1 
  2    61 ms     9 ms     3 ms  gw-vlan2593.e2.hex.ask4.net [213.143.8.25] 
  3    18 ms     4 ms     3 ms  core1.lond2.uk.ask4.net [213.143.0.150] 
  4    30 ms     4 ms     3 ms  core2.lond2.uk.ask4.net [213.143.0.145] 
  5     2 ms     2 ms     3 ms  ask4-google-10ge-1.hex.ask4.net [213.143.6.5] 
  6     4 ms     4 ms     3 ms  209.85.249.149 
  7     5 ms     5 ms     3 ms  209.85.241.95 
  8     3 ms     3 ms     5 ms  lhr25s26-in-f14.1e100.net [172.217.169.14] 

Trace complete.

The traceroute for youtube.com also begins with the same local gateway and ISP gateway as the other two routes. However, there is an early difference: at the second hop, one of the measurements shows a higher latency (61 ms), which might indicate a temporary congestion or a longer physical distance at that point.

After this initial divergence, the path for youtube.com closely resembles that of google.com. The route passes through the same or similar core routers in London and reaches the same dedicated Google network node at hop 5 (ask4-google-10ge-1.hex.ask4.net [213.143.6.5]). Hops 6 through 8 show consistent, low latencies, and the traceroute concludes at hop 8 with the IP address 172.217.169.14.

Given that YouTube is part of the Google infrastructure, it is not surprising to see a similar routing path to that of google.com. The minor differences in latency, particularly in the earlier hops, could be due to slight variations in network congestion or routing policies specific to YouTube’s traffic.